# Right Left up down spatial 7

In [5]:
import os
from pathlib import Path

import numpy as np
import tensorflow as tf
import keras
import pandas as pd

from frame_generator import FrameGenerator
from model import create_model

In [9]:
DATA_PATH = "../data_sp7_right_left_up_down_1200"
NOTEBOOK_NAME = "2_2d_plus_1_rlud_sp7"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

# PARAMS
# number of frames taken from each video
n_frames = 36
# number of frames skipped from each video
frame_step = 2
batch_size = 8
# Define the dimensions of one frame in the set of frames created
HEIGHT = 13
WIDTH = 7

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


In [7]:
from validate_files import get_video_lengths

lengths_array = get_video_lengths(f"./{DATA_PATH}/test/")
np.append(lengths_array, get_video_lengths(f"./{DATA_PATH}/train/"))
np.append(lengths_array, get_video_lengths(f"./{DATA_PATH}/validation/"))

# Calculate average
average = np.mean(lengths_array)

# Calculate maximum
maximum = np.max(lengths_array)

# Calculate minimum
minimum = np.min(lengths_array)

print("Average:", average)
print("Maximum:", maximum)
print("Minimum:", minimum)

Average: 46.21875
Maximum: 71
Minimum: 30


## Preprocess video data

Load something something data tf.data.Dataset

In [11]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['val'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=False
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['test'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        height=HEIGHT,
        width=WIDTH,
        training=False
    ),
    output_signature = output_signature
)

print(type(test_ds))

# Batch the data
test_ds = test_ds.batch(batch_size)

print(type(test_ds))


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


2024-01-04 22:10:31.606970: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/python/3.10.13/lib/python3.10/site-packages/cv2/../../lib64:
2024-01-04 22:10:31.607002: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-04 22:10:31.607031: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-5f4282): /proc/driver/nvidia/version does not exist
2024-01-04 22:10:31.607336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Model Creation

In [12]:
model = create_model(n_frames=n_frames, height=HEIGHT, width=WIDTH)

## Validate Model and Data

In [13]:
# testing tensor is setup correct
iter(train_ds)

## Build Model

In [14]:
frames, label = next(iter(train_ds))

In [15]:
model.build(frames)

## Load the Model

In [16]:
previous_runs = 0

In [17]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [18]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

## Train the Model

In [19]:
previously_run_epochs = 0
history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/50


KeyboardInterrupt: 

## Analyse results

In [18]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

for batch in test_ds:
    x, y = batch
    true_labels.extend(y.numpy())
    preds = model.predict(x)
    preds = softmax(preds, axis=1)
    preds = np.argmax(preds, axis=1)
    predictions.extend(preds)

true_labels = np.array(true_labels)
predictions = np.array(predictions)

2023-12-24 13:53:44.384908: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.
2023-12-24 13:53:45.818856: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


1/1 [==============================] - 7s 7s/step


2023-12-24 13:53:53.122601: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.
2023-12-24 13:53:53.258134: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


1/1 [==============================] - 1s 893ms/step


In [19]:
fg = FrameGenerator(
    subset_paths['test'],
    n_frames=n_frames,
    index_df=index_df,
    frame_step=frame_step,
    training=False
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

true_labels = np.array(true_labels)
predictions = np.array(predictions)

accuracy = accuracy_score(true_labels, predictions)

precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

Overall Accuracy: 0.9166666666666666
Class Pushing something from left to right: Precision: 0.9661016949152542, Recall: 0.95, F1 Score: 0.957983193277311
Class Pushing something from right to left: Precision: 0.9734513274336283, Recall: 0.9166666666666666, F1 Score: 0.944206008583691
Class Moving something up: Precision: 0.8296296296296296, Recall: 0.9333333333333333, F1 Score: 0.8784313725490196
Class Moving something down: Precision: 0.9122807017543859, Recall: 0.8666666666666667, F1 Score: 0.8888888888888888
